# Fast3r implementation

In [3]:
import torch
from fast3r.dust3r.utils.image import load_images
from fast3r.dust3r.inference_multiview import inference
from fast3r.models.fast3r import Fast3R
from fast3r.models.multiview_dust3r_module import MultiViewDUSt3RLitModule

c:\Users\gusta\Documents\ProjetosDiversao\ReconstructionStudies\.venv\Lib\site-packages\pl_bolts\__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\gusta\Documents\ProjetosDiversao\ReconstructionStudies\.venv\Lib\site-packages\lightning_fabric\__init__.py:29: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
c:\Users\gusta\Documents\ProjetosDiversao\ReconstructionStudies\.venv\Lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More deta

/!\ module trimesh is not installed, cannot visualize results /!\
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [15]:
model = Fast3R.from_pretrained("jedyang97/Fast3R_ViT_Large_512")  # If you have networking issues, try pre-download the HF checkpoint dir and change the path here to a local directory
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using device: {device}")

Using device: cpu


In [16]:
# Create a lightweight lightning module wrapper for the model.
# This provides functions to estimate camera poses, evaluate 3D reconstruction, etc.
lit_module = MultiViewDUSt3RLitModule.load_for_inference(model)

# Set model to evaluation mode
model.eval()
lit_module.eval()

# --- Load Images ---
# Provide a list of image file paths. Images can come from different cameras and aspect ratios.
filelist = "../images/"  # Change this to your image directory
images = load_images(filelist, size=512, verbose=True)

>> Loading images from ../images/
 - adding stop_01_20251128_141444.jpg with resolution 3000x4000 --> 384x512
 - adding stop_02_20251128_141452.jpg with resolution 3000x4000 --> 384x512
 - adding stop_03_20251128_141501.jpg with resolution 3000x4000 --> 384x512
 - adding stop_04_20251128_141510.jpg with resolution 3000x4000 --> 384x512
 - adding stop_05_20251128_141518.jpg with resolution 3000x4000 --> 384x512
 - adding stop_06_20251128_141527.jpg with resolution 3000x4000 --> 384x512
 - adding stop_07_20251128_141536.jpg with resolution 3000x4000 --> 384x512
 - adding stop_08_20251128_141544.jpg with resolution 3000x4000 --> 384x512
 - adding stop_09_20251128_141553.jpg with resolution 3000x4000 --> 384x512
 - adding stop_10_20251128_141602.jpg with resolution 3000x4000 --> 384x512
 - adding stop_11_20251128_141611.jpg with resolution 3000x4000 --> 384x512
 - adding stop_12_20251128_141620.jpg with resolution 3000x4000 --> 384x512
 - adding stop_13_20251128_141628.jpg with resolution 

In [20]:
# --- Run Inference ---
# The inference function returns a dictionary with predictions and view information.
output_dict, profiling_info = inference(
    images,
    model,
    device,
    dtype=torch.bfloat16,  # or use torch.bfloat16 if supported
    verbose=True,
    profiling=False,
)

>> Inference with model on 20 images


c:\Users\gusta\Documents\ProjetosDiversao\ReconstructionStudies\.venv\Lib\site-packages\torch\amp\autocast_mode.py:287: UserWarning: In CPU autocast, but the target dtype is not supported. Disabling autocast.
CPU Autocast only supports dtype of torch.bfloat16, torch.float16 currently.
  warnings.warn(error_message)
c:\Users\gusta\Documents\ProjetosDiversao\ReconstructionStudies\.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


something is wrong with the encoder, it took: 438.4850859642029


KeyError: 'dataset'

In [19]:
# --- Estimate Camera Poses ---
# This step estimates the camera-to-world (c2w) poses for each view using PnP.
poses_c2w_batch, estimated_focals = MultiViewDUSt3RLitModule.estimate_camera_poses(
    output_dict['preds'],
    niter_PnP=100,
    focal_length_estimation_method='first_view_from_global_head'
)
# poses_c2w_batch is a list; the first element contains the estimated poses for each view.
camera_poses = poses_c2w_batch[0]

# Print camera poses for all views.
for view_idx, pose in enumerate(camera_poses):
    print(f"Camera Pose for view {view_idx}:")
    print(pose.shape)  # np.array of shape (4, 4), the camera-to-world transformation matrix

# --- Extract 3D Point Clouds for Each View ---
# Each element in output_dict['preds'] corresponds to a view's point map.
for view_idx, pred in enumerate(output_dict['preds']):
    point_cloud = pred['pts3d_in_other_view'].cpu().numpy()
    print(f"Point Cloud Shape for view {view_idx}: {point_cloud.shape}")

NameError: name 'output_dict' is not defined